**Table of contents**<a id='toc0_'></a>    
- [Определение стоимости автомобилей](#toc1_)    
- [Импорт и осмотр данных](#toc2_)    
- [Подготовка данных](#toc3_)    
- [Обучение и выбор моделей](#toc4_)    
- [Анализ выбранных моделей](#toc5_)    
- [Создание итоговой модели](#toc6_)    
- [Результаты](#toc7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Определение стоимости автомобилей](#toc0_)

**Описание проблемы**

В сфере покупки и продажи автомобилей существует проблема определения точной цены для каждого конкретного автомобиля. Цена автомобиля зависит от множества факторов, таких как марка, модель, год выпуска, техническое состояние, пробег и другие характеристики. Из-за этой сложности многие автовладельцы и потенциальные покупатели сталкиваются с неопределенностью и неуверенностью в правильности оценки стоимости автомобиля.

**Цель**

Основной целью проекта является разработка модели машинного обучения, которая способна предсказывать цену автомобиля с высокой точностью на основе доступных характеристик. Это позволит упростить процесс продажи и покупки автомобилей, обеспечивая уверенность и прозрачность для всех заинтересованных сторон.

**Задачи**

1. Подготовка данных
   - Очистка данных от выбросов, ошибок или пропущенных значений.
   - Преобразование категориальных признаков в числовые (например, преобразование марки автомобиля в числовой код).
   - Масштабирование и нормализация числовых признаков (при необходимости).
  
2. Разделение данных:
   - Разделение собранных данных на обучающую выборку и тестовую выборку.
   - Задание соотношения между обучающей и тестовой выборками (например, 80% - обучающая, 20% - тестовая).

3. Выбор модели (состоит из двух этапов: выбор из множества моделей, потом остановлюсь на трех):
   - Исследование различных моделей машинного обучения, подходящих для задачи определения цены автомобиля (например, линейная регрессия, случайный лес, градиентный бустинг и т.д.).
   - Выбор наиболее подходящей модели на основе метрики RMSE


4. Обучение модели:
   - Обучение выбранной модели на обучающей выборке.
   - Тюнинг гиперпараметров модели для достижения наилучшей производительности.
   - Оценка модели на тестовой выборке для проверки ее обобщающей способности.

# <a id='toc2_'></a>[Импорт и осмотр данных](#toc0_)

<font color='orange'>Библиотеки и данные</font>

In [137]:
# Pandas
import pandas as pd

# Numpy
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import LinearSVR
from sklearn.compose import make_column_transformer

# LightGMB
from lightgbm import LGBMRegressor

# Catboost
from catboost import CatBoostRegressor

# Time
import time

In [138]:
data = pd.read_csv('/kaggle/input/cars-prices/autos.csv')

<font color='orange'>Осмотр данных</font>

In [139]:
display(data.head(3))

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46


<font color='orange'>Размер и пропуски и дубликаты</font>

In [140]:
print(data.shape)
print(data.dropna().shape)
print(data.duplicated().sum())
print(data.isna().sum())

(354369, 16)
(245814, 16)
4
DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64


<font color='orange'>Временной период который представляют данные</font>

In [141]:
print((pd.to_datetime(data['DateCreated'])).dt.year.unique())
print((pd.to_datetime(data['DateCrawled'])).dt.year.unique())
print((pd.to_datetime(data['LastSeen'])).dt.year.unique())
print(sorted((pd.to_datetime(data['DateCreated'])).dt.month.unique()))

[2016 2015 2014]
[2016]
[2016]
[1, 2, 3, 4, 6, 8, 9, 11, 12]


<font color='orange'>Типы данных</font>

In [142]:
data.dtypes

DateCrawled          object
Price                 int64
VehicleType          object
RegistrationYear      int64
Gearbox              object
Power                 int64
Model                object
Kilometer             int64
RegistrationMonth     int64
FuelType             object
Brand                object
Repaired             object
DateCreated          object
NumberOfPictures      int64
PostalCode            int64
LastSeen             object
dtype: object

<font color='orange'>Модели машин</font>

In [143]:
print(np.sort(data['Model'].dropna().unique()))

['100' '145' '147' '156' '159' '1_reihe' '1er' '200' '2_reihe' '300c'
 '3_reihe' '3er' '4_reihe' '500' '5_reihe' '5er' '601' '6_reihe' '6er'
 '7er' '80' '850' '90' '900' '9000' '911' 'a1' 'a2' 'a3' 'a4' 'a5' 'a6'
 'a8' 'a_klasse' 'accord' 'agila' 'alhambra' 'almera' 'altea' 'amarok'
 'antara' 'arosa' 'astra' 'auris' 'avensis' 'aveo' 'aygo' 'b_klasse'
 'b_max' 'beetle' 'berlingo' 'bora' 'boxster' 'bravo' 'c1' 'c2' 'c3' 'c4'
 'c5' 'c_klasse' 'c_max' 'c_reihe' 'caddy' 'calibra' 'captiva' 'carisma'
 'carnival' 'cayenne' 'cc' 'ceed' 'charade' 'cherokee' 'citigo' 'civic'
 'cl' 'clio' 'clk' 'clubman' 'colt' 'combo' 'cooper' 'cordoba' 'corolla'
 'corsa' 'cr_reihe' 'croma' 'crossfire' 'cuore' 'cx_reihe' 'defender'
 'delta' 'discovery' 'doblo' 'ducato' 'duster' 'e_klasse' 'elefantino'
 'eos' 'escort' 'espace' 'exeo' 'fabia' 'fiesta' 'focus' 'forester'
 'forfour' 'fortwo' 'fox' 'freelander' 'fusion' 'g_klasse' 'galant'
 'galaxy' 'getz' 'gl' 'glk' 'golf' 'grand' 'i3' 'i_reihe' 'ibiza'
 'impreza' '

<font color='orange'>Бренды</font>

In [144]:
print(data['Brand'].unique())

['volkswagen' 'audi' 'jeep' 'skoda' 'bmw' 'peugeot' 'ford' 'mazda'
 'nissan' 'renault' 'mercedes_benz' 'opel' 'seat' 'citroen' 'honda' 'fiat'
 'mini' 'smart' 'hyundai' 'sonstige_autos' 'alfa_romeo' 'subaru' 'volvo'
 'mitsubishi' 'kia' 'suzuki' 'lancia' 'toyota' 'chevrolet' 'dacia'
 'daihatsu' 'trabant' 'saab' 'chrysler' 'jaguar' 'daewoo' 'porsche'
 'rover' 'land_rover' 'lada']


<font color='orange'>Кузов</font>

In [145]:
print(data['VehicleType'].unique())

[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']


<font color='orange'>Тип топлива</font>

In [146]:
print(data['FuelType'].unique())

['petrol' 'gasoline' nan 'lpg' 'other' 'hybrid' 'cng' 'electric']


<font color='orange'>Осмотр численных значений</font>

In [147]:
print(data[['Price', 'Power', 'Kilometer']].describe())
print(np.sort(data['RegistrationMonth'].unique()))
print(np.sort(data['RegistrationYear'].unique()))

               Price          Power      Kilometer
count  354369.000000  354369.000000  354369.000000
mean     4416.656776     110.094337  128211.172535
std      4514.158514     189.850405   37905.341530
min         0.000000       0.000000    5000.000000
25%      1050.000000      69.000000  125000.000000
50%      2700.000000     105.000000  150000.000000
75%      6400.000000     143.000000  150000.000000
max     20000.000000   20000.000000  150000.000000
[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
[1000 1001 1039 1111 1200 1234 1253 1255 1300 1400 1500 1600 1602 1688
 1800 1910 1915 1919 1920 1923 1925 1927 1928 1929 1930 1931 1932 1933
 1934 1935 1936 1937 1938 1940 1941 1942 1943 1944 1945 1946 1947 1948
 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962
 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976
 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990
 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
 20

<style>
    .bordered-cell {
        border: 3px solid black;
        padding: 10px;
    }
</style>

<div class="bordered-cell">

<font color='orange'>**Вывод**</font>

- Данных много (350к строк), без пропусков (250к строк)
- Столбцы переименую
- Есть лишние столбцы (даты и время, почтовый индекс, количество фото)
- Данные практически только за 2016 год
- Столбец с месяцем регистрации содержит 13 (!) месяцев
- Столбец с годом регистрации нужно почистить и оставить актуальные машины
- Столбец с ценой содержит цены равные нулю
- Столбец с мощностью содержит и нулевые и нереально высокие мощности
- Столбцы с моделью и брендом машины вроде норм
- 4 дубликата - удалю
- Пропуски в столбцах модели, топлива, коробки передач, типа кузова заполню если другие столбцы есть исходя из других данных
- пропуски в строке с ремонтом заполню ответами "нет", вероятно их не заполняли если не было ремонта

# <a id='toc3_'></a>[Подготовка данных](#toc0_)

<font color='orange'>Переименование столбцов</font>

In [148]:
new_columns = ['DateCrawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
               'power', 'model', 'kilometer', 'registration_month', 'fuel_type', 'brand',
               'repaired', 'DateCreated', 'NumberOfPictures', 'PostalCode',
               'LastSeen']
data.columns = new_columns

<font color='orange'>Удаление лишних столбцов</font>

In [149]:
data.drop(['DateCrawled', 'DateCreated', 'NumberOfPictures',
          'PostalCode', 'LastSeen'], axis=1, inplace=True)

<font color='orange'>Месяц регистрации</font>

Тут возникла проблема, месяц регистрации не ясен. Сначала я считал что 12-й месяц это неправильный или 0-ой месяц. Но они содержат много значений. Возможно, если есть связь с источником данных то можно уточнить, какой номер месяца является допустим "отсутствием" месяца регистрации. Но неясно. Поэтому пока оставлю как есть. Либо попробую удалить из датасета потом.

In [150]:
print((data['registration_month'] == 12).sum())
print((data['registration_month'] == 0).sum())

24289
37352


<font color='orange'>Год регистрации</font>

Удаляю строки с значениями больше 2016, так как датасет датируется 2016 годом.
Удаляю строки с значениями меньше 1980, так как возможно более старые машины будут "раритетом" и цены в модели будет расчитыватся неправильно. Представим, что заказчику нужны цены на актуальные модели, а оценку "раритетов" оставим профессионалам) 

In [151]:
data = data[data['registration_year'] < 2016]
data = data[data['registration_year'] > 1980]

<font color='orange'>Цена</font>

Машины с ценой ниже 50 (евро) считаю выбросами и выбрасываю. Для создания модели они будут не нужны

In [152]:
data = data[data['price'] > 50]

<font color='orange'>Мощность</font>

Самые мощние серийные автомобили недотягивают до 2000 ЛС. Поэтому оставлю все машины меньше 2000 ЛС и больше 10 ЛС

In [153]:
data = data[data['power'] < 2000]
data = data[data['power'] > 10]

<font color='orange'>Дубликаты</font> А вот и дубликаты. Видимо после удаления столбцов с датами, нашлись дубликаты объявлений

In [154]:
data.duplicated().sum()
data = data.drop_duplicates()

<font color='orange'>Модель</font> Удаляю модели "other"

In [155]:
data = data[data['model'] != "other"]

data.shape

(246076, 11)

<font color='orange'>Пропуски</font>

Интересная задача - востановить данные в пропусках в топливе, коробке, модели, кузове исходя из других столбцов. А те что невозможно восстановить - удалю.

<font color='orange'>Пропуски модели</font>
Самое простое - модель узнать невозомжно. Удаляю.

In [156]:
data.dropna(subset=['model'], inplace=True)

<font color='orange'>Пропуски тип кузова</font> Тип кузова можно узнать из модели. 

- Первое, что я хотел сделать, просто удалить этот столбец. Логика: тип кузова определяться моделью и это лишний признак
- Но у одной модели может быть несколько модификаций, в том числе с разным кузовом, поэтому лучше оставить
- В датасете не даны конкретные названия моделей, а только общие
- Потом я хотел заполнить столбы при помощи модели решающего дерева, применив OHE для столбца модель и сделать целевой признак - тип кузова. Но закрались подозрения по поводу результата предсказания для этого решения
- В итоге решил заполнить самым частым значением кузова для модели

In [157]:
# Пример значений до заполнения
print(data[['model', 'vehicle_type']][data['model'] == 'golf'].head(6))

    model vehicle_type
3    golf        small
9    golf        small
58   golf        sedan
74   golf        sedan
77   golf          NaN
113  golf        sedan


In [158]:
# группировка по модели и сбор всех значений типа кузова для каждой модели
model_vehicle_type_dict = {}
for model, group in data.groupby('model'):
    bodies = group['vehicle_type'].dropna().tolist()
    if len(bodies) > 0:
        model_vehicle_type_dict[model] = bodies

# функция для заполнения пропущенных значений типа кузова


def fill_missing_body(row, model_vehicle_type_dict):
    if pd.isnull(row['vehicle_type']):
        model = row['model']
        if model in model_vehicle_type_dict:
            bodies = model_vehicle_type_dict[model]
            if len(bodies) > 0:
                most_common_body = max(set(bodies), key=bodies.count)
                return most_common_body
    return row['vehicle_type']


# применение функции ко всем строкам датафрейма
data['vehicle_type'] = data.apply(
    lambda row: fill_missing_body(row, model_vehicle_type_dict), axis=1)

In [159]:
# Пример значений после заполнения
print(data[['model', 'vehicle_type']][data['model'] == 'golf'].head(6))

    model vehicle_type
3    golf        small
9    golf        small
58   golf        sedan
74   golf        sedan
77   golf        sedan
113  golf        sedan


<font color='orange'>Пропуски коробка передач</font> Тут тоже просто, коробки передач могут быть разными у одной модели, если заполнить их наиболее встречаемыми, то данные будут некачественными. Догадаться о них невозможно, только обучая модель для определения коробки, но вот сомнительный вариант. Удалю строки с пропусками

In [160]:
data.dropna(subset=['gearbox'], inplace=True)

<font color='orange'>Пропуски топливо</font> С топливом та же история, модель может иметь разные двигатели и топливо. Поэтому для качества данных лучше сохранить только исходные значения

In [161]:
data.dropna(subset=['fuel_type'], inplace=True)

<font color='orange'>Пропуски ремонт</font> Заполню пропуски значениями "не были в ремонте" и заменю на значения числовые

In [162]:
data['repaired'].fillna('no', inplace=True)
data['repaired'] = data['repaired'].replace({"yes": 1, "no": 0})
print(data['repaired'].value_counts())

0    202886
1     21114
Name: repaired, dtype: int64


<font color='orange'>Проверка обработки</font>

In [163]:
print(data.duplicated().sum())
print(data.isna().sum().sum())

2922
0


<font color='orange'>Удаляю дубликаты еще раз</font>
 
Опять появились дубликаты, видимо это одни и те же обьявления но в разное время либо на разных сайтах.

In [164]:
data = data.drop_duplicates()

<font color='orange'>Размеры и общая информация</font>

In [165]:
print(data.shape)
print(data.dtypes)

(221078, 11)
price                  int64
vehicle_type          object
registration_year      int64
gearbox               object
power                  int64
model                 object
kilometer              int64
registration_month     int64
fuel_type             object
brand                 object
repaired               int64
dtype: object


<font color='orange'>Создаю данные кодированные OHE в отдельной переменной</font>

In [166]:
features_list_OHE = ['vehicle_type', 'gearbox',
                     'fuel_type', 'brand']
features_list_OE = ['model']
features_numerical = ['registration_year',
                      'power', 'kilometer', 'registration_month']

In [167]:
col_transformer = make_column_transformer(
    (
        OneHotEncoder(drop='first', handle_unknown='error', sparse=False),
        features_list_OHE
    ),
    (
        OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1),
        features_list_OE
    ),
    (
        StandardScaler(),
        features_numerical
    ),
    remainder='passthrough'
)

<font color='orange'>Разбитие на трейн/валид/тест</font>

In [168]:
data_2 = data.copy()

target = data['price']
features = data.drop(['price'], axis=1)
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

<font color='orange'>Масштабирование + кодирование</font>

In [169]:
col_transformer.fit(features_train)
features_train = col_transformer.transform(features_train)
features_valid = col_transformer.transform(features_valid)
features_test = col_transformer.transform(features_test)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


<style>
    .bordered-cell {
        border: 3px solid black;
        padding: 10px;
    }
</style>

<div class="bordered-cell">

<font color='orange'>**ВЫВОД**</font>
1. Результать обработки датасета
   - Осталось 234 тысячи строк
   - Дубликатов нет
   - Пропусков нет
   - Лишние столбцы удалены
   - Выбросы удалены
   - Отдельно перекодировал признаки в OHE и в OE
   - Данные масштабированы
2. Данные перекодированы и разбиты на train, valid, test - Первый вариант данных. (features_valid, features_test, target_valid, target_test, features_train, target_train)
3. Отдельно сохранил исходные данные (но эти данные тоже обработаны, но без кодировки признаков) - для использовния в моделях градиентного бустинга. Данные находятся в переменной data_2

# <a id='toc4_'></a>[Обучение и выбор моделей](#toc0_)

Две части:
1. Проведу быстрое обучение моделей без подбора гиперпараметров
2. На трех-четырех лучших подберу гиперпараметры моделей


<font color='orange'>Предварительная проверка dummy модели</font> 

In [170]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(features_train, target_train)
prediction_dummy = dummy_regr.predict(features_valid)
print("RMSE (valid) =", mean_squared_error(
    target_valid, prediction_dummy)**0.5)

RMSE (valid) = 4648.597866258329


<font color='orange'>Модели для обучения:</font> 

1. Линейная регрессия (Sklearn)
2. Модель Lasso (Sklearn)
3. Модель Ridge (Sklearn)
4. Модель ElasticNet (Sklearn)
5. Support Vector Machine (Sklearn)
6. Решающее дерево (Sklearn)
7. Градиентный бустинг (LightGBM)
8. Градиентный бустинг (Catboost)

<font color='orange'>Линейная регрессия</font> 

In [171]:
%%time

model = LinearRegression()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("RMSE (valid) =", mean_squared_error(target_valid, predicted_valid)**0.5)

RMSE (valid) = 2509.2891488447876
CPU times: user 921 ms, sys: 210 ms, total: 1.13 s
Wall time: 519 ms


<font color='orange'>Модель Lasso</font> 

In [172]:
%%time

model = LassoCV()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("RMSE (valid) =", mean_squared_error(target_valid, predicted_valid)**0.5)

RMSE (valid) = 2538.6453672358857
CPU times: user 4.66 s, sys: 2.82 s, total: 7.48 s
Wall time: 2.61 s


<font color='orange'>Модель Ridge</font> 

In [173]:
%%time

model = Ridge()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("RMSE (valid) =", mean_squared_error(target_valid, predicted_valid)**0.5)

RMSE (valid) = 2509.2697578078437
CPU times: user 278 ms, sys: 262 ms, total: 540 ms
Wall time: 176 ms


<font color='orange'>Модель ElasticNet</font> 

In [174]:
%%time

model = ElasticNetCV()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("RMSE (valid) =", mean_squared_error(target_valid, predicted_valid)**0.5)

RMSE (valid) = 4146.630805326385
CPU times: user 4.18 s, sys: 2.79 s, total: 6.97 s
Wall time: 2.45 s


<font color='orange'>Support Vector Machine</font> 

In [175]:
%%time

model = LinearSVR()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("RMSE (valid) =", mean_squared_error(target_valid, predicted_valid)**0.5)

RMSE (valid) = 2717.005524826384
CPU times: user 4.93 s, sys: 224 ms, total: 5.15 s
Wall time: 4.83 s


<font color='orange'>Решающее дерево</font> 

In [176]:
%%time

model = DecisionTreeRegressor()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("RMSE (valid) =", mean_squared_error(target_valid, predicted_valid)**0.5)

RMSE (valid) = 1997.2407298227192
CPU times: user 1.74 s, sys: 197 ms, total: 1.94 s
Wall time: 1.63 s


<font color='orange'>Градиентный бустинг (LightGBM)</font> 

In [177]:
%%time

model = LGBMRegressor()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("RMSE (valid) =", mean_squared_error(target_valid, predicted_valid)**0.5)

RMSE (valid) = 1566.0427666587937
CPU times: user 3.45 s, sys: 468 ms, total: 3.92 s
Wall time: 1.92 s


<font color='orange'>Градиентный бустинг (Catboost)</font> 

In [178]:
%%time

model = CatBoostRegressor(logging_level='Silent')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("RMSE (valid) =", mean_squared_error(target_valid, predicted_valid)**0.5)

RMSE (valid) = 1492.809297357367
CPU times: user 51.8 s, sys: 2.69 s, total: 54.5 s
Wall time: 18.5 s


<style>
    .bordered-cell {
        border: 3px solid black;
        padding: 10px;
    }
</style>

<div class="bordered-cell">

<font color='orange'>Промежуточный вывод (Выбираю три лучшие)</font>

Больше всего понравились модели по метрике и скорости обучения:

1. Catboost
2. LightGBM
3. Ridge (хотя он практически одинаков с Lasso, но скорость выше)


- ElasticNet, Lasso, Ridge, SVM примерно одинкавы
- У константной модели RMSE около 4500, что в 2-3 раза хуже чем у любой другой модели

<font color='orange'>Ridge с поиском гиперпараметров</font>

In [179]:
%%time

model = Ridge()
CV = 4
params = {'alpha': [10, 1, 0.1, 0.01],
                "fit_intercept": [True, False]}

model_GS = GridSearchCV(model, params, cv = CV, scoring='neg_root_mean_squared_error', n_jobs=-1)

model_GS.fit(features_train, target_train)
predicted_valid = model_GS.predict(features_valid)
n_iterations = len(model_GS.cv_results_['params'])
avg_time = model_GS.cv_results_['mean_fit_time'].mean()
total_time = model_GS.cv_results_['mean_fit_time'].sum() * CV


start_time = time.time()
model_GS.predict(features_valid)
predict_time = time.time() - start_time
print(f'predict_time {predict_time:.2f}')


print(f'Ridge model, params = {model_GS.best_params_}, score = {-model_GS.best_score_}, avg time {avg_time:.1f}, total time {total_time:.1f}')
ridge = [model_GS.best_params_, -model_GS.best_score_, avg_time, total_time, n_iterations, predict_time]

predict_time 0.00
Ridge model, params = {'alpha': 1, 'fit_intercept': True}, score = 2515.417693347794, avg time 0.1, total time 4.4
CPU times: user 9.64 s, sys: 7.86 s, total: 17.5 s
Wall time: 4.78 s


<font color='orange'>LightGBM с поиском гиперпараметров</font>

Подготавливаю данные без кодировки признаков (суффикс - **_raw**)

In [180]:
target_raw = data_2['price']
features_raw = data_2.drop(['price'], axis=1)

In [181]:
features_raw[features_list_OHE] = features_raw[features_list_OHE].astype(
    'category')
features_raw['model'] = features_raw['model'].astype('category')

features_train_raw, features_valid_test_raw, target_train_raw, target_valid_test_raw = train_test_split(
    features_raw, target_raw, test_size=0.4, random_state=12345)
features_valid_raw, features_test_raw, target_valid_raw, target_test_raw = train_test_split(
    features_valid_test_raw, target_valid_test_raw, test_size=0.5, random_state=12345)

In [182]:
features_raw.dtypes

vehicle_type          category
registration_year        int64
gearbox               category
power                    int64
model                 category
kilometer                int64
registration_month       int64
fuel_type             category
brand                 category
repaired                 int64
dtype: object

Модель LGBMRegressor на данных без кодировки

In [183]:
%%time

model = LGBMRegressor()
CV = 3
params = {'learning_rate' : [0.01, 0.1, 0.5]}

model_GS = GridSearchCV(model, params, cv = CV, scoring='neg_root_mean_squared_error', n_jobs=-1)

model_GS.fit(features_train_raw, target_train_raw)
predicted_valid = model_GS.predict(features_valid_raw)
n_iterations = len(model_GS.cv_results_['params'])*CV
avg_time = model_GS.cv_results_['mean_fit_time'].mean()
total_time = model_GS.cv_results_['mean_fit_time'].sum() * CV

start_time = time.time()
model_GS.predict(features_valid_raw)
predict_time = time.time() - start_time
print(predict_time)

print(f'LGBMRegressor model, params = {model_GS.best_params_}, score = {-model_GS.best_score_}, avg time {avg_time:.1f}, total time {total_time:.1f}')
light2 = [model_GS.best_params_, -model_GS.best_score_, avg_time, total_time, n_iterations, predict_time]

0.16587495803833008
LGBMRegressor model, params = {'learning_rate': 0.1}, score = 1497.4839264009945, avg time 1.3, total time 11.3
CPU times: user 27.6 s, sys: 4.68 s, total: 32.2 s
Wall time: 14.2 s


In [184]:
%%time

model = LGBMRegressor()
CV = 3
params = {'learning_rate' : [0.01, 0.1, 0.5]}

model_GS = GridSearchCV(model, params, cv = CV, scoring='neg_root_mean_squared_error', n_jobs=-1)

model_GS.fit(features_train, target_train)
predicted_valid = model_GS.predict(features_valid)
n_iterations = len(model_GS.cv_results_['params'])*CV
avg_time = model_GS.cv_results_['mean_fit_time'].mean()
total_time = model_GS.cv_results_['mean_fit_time'].sum() * CV

start_time = time.time()
model_GS.predict(features_valid)
predict_time = time.time() - start_time
print(predict_time)

print(f'LGBMRegressor model, params = {model_GS.best_params_}, score = {-model_GS.best_score_}, avg time {avg_time:.1f}, total time {total_time:.1f}')
light = [model_GS.best_params_, -model_GS.best_score_, avg_time, total_time, n_iterations, predict_time]

0.07609033584594727
LGBMRegressor model, params = {'learning_rate': 0.5}, score = 1529.9399719409078, avg time 1.2, total time 11.2
CPU times: user 26.6 s, sys: 4.88 s, total: 31.4 s
Wall time: 13.5 s


<font color='orange'>Catboost с поиском гиперпараметров</font>

In [185]:
%%time

model = CatBoostRegressor(logging_level='Silent')
CV = 3
params = {'learning_rate' : [0.01, 0.02, 0.05]}

model_GS = GridSearchCV(model, params, cv = CV, 
scoring='neg_root_mean_squared_error', n_jobs=-1)

model_GS.fit(features_train, target_train)
predicted_valid = model_GS.predict(features_valid)
n_iterations = len(model_GS.cv_results_['params'])*CV
avg_time = model_GS.cv_results_['mean_fit_time'].mean()
total_time = model_GS.cv_results_['mean_fit_time'].sum() * CV

start_time = time.time()
model_GS.predict(features_valid)
predict_time = time.time() - start_time
print(predict_time)

print(f'CatBoostRegressor model, params = {model_GS.best_params_}, score = {-model_GS.best_score_}, avg time {avg_time:.1f}, total time {total_time:.1f}')
catboost = [model_GS.best_params_, -model_GS.best_score_, avg_time, total_time, n_iterations, predict_time]

1.4460525512695312
CatBoostRegressor model, params = {'learning_rate': 0.05}, score = 1509.9034111724834, avg time 12.1, total time 109.0
CPU times: user 6min 25s, sys: 22.3 s, total: 6min 47s
Wall time: 2min 22s


In [186]:
%%time

cat_cols = ['brand', 'model', 'vehicle_type', 'fuel_type', 'gearbox', 'repaired']
model = CatBoostRegressor(cat_features=cat_cols, logging_level='Silent')
CV = 3
params = {'learning_rate' : [0.01, 0.05, 0.1]}

model_GS = GridSearchCV(model, params, cv = CV, scoring='neg_root_mean_squared_error', n_jobs=-1)

model_GS.fit(features_train_raw, target_train_raw)
predicted_valid = model_GS.predict(features_valid_raw)
n_iterations = len(model_GS.cv_results_['params'])*CV
avg_time = model_GS.cv_results_['mean_fit_time'].mean()
total_time = model_GS.cv_results_['mean_fit_time'].sum() * CV

start_time = time.time()
model_GS.predict(features_valid_raw)
predict_time = time.time() - start_time
print(predict_time)

print(f'CatBoostRegressor model, params = {model_GS.best_params_}, score = {-model_GS.best_score_}, avg time {avg_time:.1f}, total time {total_time:.1f}')
catboost2 = [model_GS.best_params_, -model_GS.best_score_, avg_time, total_time, n_iterations, predict_time]

0.41727519035339355
CatBoostRegressor model, params = {'learning_rate': 0.1}, score = 1482.9986844294963, avg time 51.1, total time 459.6
CPU times: user 32min 38s, sys: 1min 5s, total: 33min 43s
Wall time: 8min 57s


# <a id='toc5_'></a>[Анализ выбранных моделей](#toc0_)

<font color='orange'>Метрики выбранных моделей</font>

In [187]:
metrics_df = pd.DataFrame({'Ridge': ridge,
                           'LightGBM': light, 'LightGBM_raw_data': light2,
                           'CatBoost': catboost,'CatBoost_raw_data': catboost2
                           },
                          index=['best_params', 'best_score', 'avg_time', 'total_time', 'n_iterations', 'predict_time'])

display(metrics_df)

,Ridge,LightGBM,LightGBM_raw_data,CatBoost,CatBoost_raw_data
best_params,"{'alpha': 1, 'fit_intercept': True}",{'learning_rate': 0.5},{'learning_rate': 0.1},{'learning_rate': 0.05},{'learning_rate': 0.1}
best_score,2515.417693,1529.939972,1497.483926,1509.903411,1482.998684
avg_time,0.137791,1.249292,1.259443,12.109305,51.070168
total_time,4.409301,11.243629,11.334986,108.983745,459.631509
n_iterations,8,9,9,9,9
predict_time,0.004332,0.07609,0.165875,1.446053,0.417275


<style>
    .bordered-cell {
        border: 3px solid black;
        padding: 10px;
    }
</style>

<div class="bordered-cell">

<font color='orange'>**Вывод**</font>

- Catboost и LightGBM практически одинаковы по RMSE
- LightGBM обучается быстрее, но это зависит от выбора гиперпараметров.
- LightGBM быстрее предсказывает.
- **Данные без кодировки в моделях градиентного бустинга улучшают метрику RMSE в данноей задаче, по сравнению с кодированными признаками**
- Выбираю catboost для финальной модели и данные без кодировки

# <a id='toc6_'></a>[Создание итоговой модели](#toc0_)

In [188]:
# Объединяю в валид и трейн в новый трейн
features_new = pd.concat([features_train_raw, features_valid_raw])
target_new = pd.concat([target_train, target_valid])

In [189]:
start_time = time.time()

model = CatBoostRegressor(depth=10, iterations=1000,
                          learning_rate=0.1, verbose=100,
                          cat_features=cat_cols)
model.fit(features_new, target_new)
predicted_test = model.predict(features_test_raw)
print("RMSE (test) =", mean_squared_error(target_test, predicted_test)**0.5)

fit_time = time.time() - start_time

start_time = time.time()
model.predict(features_test_raw)
predict_time = time.time() - start_time

print(f'fit time {fit_time}, predict_time {predict_time}')

0:	learn: 4306.2004638	total: 237ms	remaining: 3m 56s
100:	learn: 1479.8164587	total: 21.5s	remaining: 3m 11s
200:	learn: 1407.6968387	total: 43.3s	remaining: 2m 52s
300:	learn: 1364.9641024	total: 1m 5s	remaining: 2m 32s
400:	learn: 1337.9282798	total: 1m 28s	remaining: 2m 11s
500:	learn: 1312.1859643	total: 1m 51s	remaining: 1m 50s
600:	learn: 1290.4563264	total: 2m 14s	remaining: 1m 29s
700:	learn: 1271.7738726	total: 2m 37s	remaining: 1m 7s
800:	learn: 1254.7370865	total: 3m 1s	remaining: 45s
900:	learn: 1240.9691630	total: 3m 24s	remaining: 22.5s
999:	learn: 1227.3614020	total: 3m 47s	remaining: 0us
RMSE (test) = 1441.4093325832011
fit time 232.25557708740234, predict_time 0.8732967376708984


In [190]:
importances = model.feature_importances_
feature_importance = pd.DataFrame({'importance': importances, 'features': features_train_raw.columns}).sort_values(
    'importance', ascending=False).reset_index(drop=True)
feature_importance.head(10)

,importance,features
0,32.246634,registration_year
1,22.257239,power
2,14.142230,brand
3,8.441077,vehicle_type
4,7.776901,kilometer
5,7.084540,model
6,2.868199,fuel_type
7,2.460238,repaired
8,1.540894,registration_month
9,1.182047,gearbox


# <a id='toc7_'></a>[Результаты](#toc0_)

<style>
    .bordered-cell {
        border: 3px solid black;
        padding: 10px;
    }
</style>

<div class="bordered-cell">

<font color='orange'>**Модель**</font>

1. Построил модель на основе Catboost
2. RMSE равно 1441 евро
3. Время обучения 232.6 сек
4. Время предсказания 0.83 сек
5. Модель еще можно улучшить - категоризацию по километражу добавить, возможно преобразвать признаки, поискать новые признаки (например сколько владельцев было, комплектацию машин и т.д)
 
<font color='orange'>**Общий вывод**</font>

1. Градиентный бустинг лучший для данного проекта
2. Модель (catboost) обученная на некодированных данных работает чуть лучше, чем модель обученная на кодировке OHE, OE
3. Важность признаков (по убыванию):
    - registration_year
    - power
    - brand
    - vehicle_type
    - kilometer
    - model
    - fuel_type
    - repaired
    - registration_month
    - gearbox